In [ ]:
import io
import requests

from PIL import Image as im

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

In [ ]:
import pandas as pd

pdf = pd.read_parquet('../../ftransfer_ztf_2024-02-01_689626')


In [ ]:
pdf

In [ ]:
Id = pdf['objectId'][1]

id_plus_repete = pdf['objectId'].value_counts().idxmax()

# We select data based on their shared ID, (here opting for the most frequently occurring alert).
pdf_selectionne = pdf.loc[pdf['objectId'] == id_plus_repete]



In [ ]:
# Selecting the final alert ( if the data is ordered).
pdf_selectionne = pdf_selectionne.iloc[-1]

In [ ]:
pdf_selectionne['objectId']

In [ ]:
pdf_selectionne_cand = pdf_selectionne['prv_candidates'] 

In [ ]:
#  'candidate' the actual value 
keys = pdf_selectionne_cand[0].keys()
actual_cand = {key: pdf_selectionne['candidate'][key] for key in keys if key in pdf_selectionne['candidate']}


In [ ]:
liste_dicts = list(pdf_selectionne_cand)
liste_dicts.append(actual_cand)
df = pd.DataFrame(liste_dicts)

In [ ]:
df

In [ ]:

mjd = df['jd'].apply(lambda x: x - 2400000.5)

fig = plt.figure(figsize=(25, 15))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}

# valid values 
maskValid = (df['rb'] > 0.55) & (df['nbad'] == 0)
# Upper limit values
maskUpper = pd.isna(df['magpsf'])
#bad quality values 
maskBadquality = ~maskValid & ~maskUpper


for filt in np.unique(df['fid']):
    maskFilt = df['fid'] == filt

    plt.errorbar(
        df[maskValid & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskValid & maskFilt]['magpsf'],
        df[maskValid & maskFilt]['sigmapsf'],
        ls = '', marker='o', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )

    plt.plot(
        df[maskUpper & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskUpper & maskFilt]['diffmaglim'],
        ls='', marker='v', color=colordic[filt], markerfacecolor='none'
    )
    

    plt.errorbar(
        df[maskBadquality & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskBadquality & maskFilt]['magpsf'],
        df[maskBadquality & maskFilt]['sigmapsf'],
        ls='', marker='^', color=colordic[filt]
    )

plt.ylim(12, 22)
plt.gca().invert_yaxis()
plt.legend()
plt.title('Difference image PSF-fit magnitude')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Difference Magnitude');


In [ ]:

 

fig = plt.figure(figsize=(15, 5))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}
    
maskValid = (df['rb'] > 0.55) & (df['nbad'] == 0)

    
#t or 1 => candidate is from positive (sci minus ref) subtraction;
#f or 0 => candidate is from negative (ref minus sci) subtraction"
maskneg = (df['isdiffpos'] == 't') | (df['isdiffpos'] == 1)
maskpos = (df['isdiffpos'] == 'f') | (df['isdiffpos'] == 0)

for filt in np.unique(df['fid']):
    maskFilt = df['fid'] == filt

    # candidates from negative 

    plt.errorbar(
        df[maskValid & maskFilt & maskneg ]['jd'].apply(lambda x: x - 2400000.5),
        df[maskValid & maskFilt & maskneg ]['magpsf'],
        df[maskValid & maskFilt & maskneg ]['sigmapsf'],
        ls = '', marker='o', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )
    
    # candidates from positive 
    plt.errorbar(
        df[maskValid & maskFilt &  maskpos ]['jd'].apply(lambda x: x - 2400000.5),
        df[maskValid & maskFilt &  maskpos ]['magpsf'],
        df[maskValid & maskFilt &  maskpos ]['sigmapsf'],
        ls = '', marker='^', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )
    

plt.ylim(12, 22)
plt.gca().invert_yaxis()
plt.legend()
plt.title('Difference image PSF-fit magnitude')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Difference Magnitude');



In [ ]:
# from fink_science.conversion import dc_mag
from fink_utils.photometry.conversion import dc_mag
from fink_utils.photometry.utils import is_source_behind

# Take only valid measurements
maskValid = (df['rb'] > 0.55) & (df['nbad'] == 0)
df_valid = df[maskValid].sort_values('jd', ascending=False)

isSource = is_source_behind(
    df_valid['distnr'].values[0]
)

if isSource:
    print('It looks like there is a source behind. Lets compute the DC magnitude instead.')
    
    # Use DC magnitude instead of difference mag
    mag_dc, err_dc = np.transpose(
        [
            dc_mag(*args) for args in zip(
                df_valid['magpsf'].astype(float).values,
                df_valid['sigmapsf'].astype(float).values,
                df_valid['magnr'].astype(float).values,
                df_valid['sigmagnr'].astype(float).values,
                df_valid['isdiffpos'].values
            )
        ]
    )
    
    df_valid['mag_dc'] = mag_dc
    df_valid['err_dc'] = err_dc
else:
    print('No source found -- keeping PSF fit magnitude')
    df_valid['mag_dc'] = df_valid['magpsf']
    df_valid['err_dc'] = df_valid['sigmapsf']



In [ ]:
fig = plt.figure(figsize=(15, 5))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}

for filt in np.unique(df_valid['fid']):
    maskFilt = df_valid['fid'] == filt

    plt.errorbar(
        df_valid[maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df_valid[maskFilt]['magpsf'],
        df_valid[maskFilt]['sigmapsf'],
        ls = '', marker='x', 
        color=colordic[filt], 
        label='{} band (PSF-fit)'.format(filtdic[filt]),
    )
    
    
    plt.errorbar(
        df_valid[maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df_valid[maskFilt]['mag_dc'],
        df_valid[maskFilt]['err_dc'],
        ls = '', marker='o', 
        color=colordic[filt], 
        label='{} band (DC)'.format(filtdic[filt]),
    )
    #To show if there is a variance in the reference
    plt.errorbar(
        df_valid[maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df_valid[maskFilt]['magnr'],
        ls = '--', 
        color=colordic[filt], 
        label='{} ref (DC)'.format(filtdic[filt]),
    )
    

ref_r = df_valid['magnr']*(df_valid['fid']==2)

ref_r = (ref_r[ref_r != 0]).mean()

ref_g = df_valid['magnr']*(df_valid['fid']==1)

ref_g = (ref_g[ref_g != 0]).mean()


print(ref_r, ref_g)
#  Average Values (Used in Fink)
#plt.axhline(y=ref_r, color=colordic[2], linestyle='--')
#plt.axhline(y=ref_g, color=colordic[1], linestyle='--')
plt.gca().invert_yaxis()
plt.legend()
plt.title('Comparison of PSF-fit and DC magnitudes')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Magnitude');

In [ ]:
fig = plt.figure(figsize=(15, 7))

from fink_utils.photometry.conversion import apparent_flux


dc_flux, dc_sigflux = np.transpose(
        [
            apparent_flux(*args, jansky=True) for args in zip(
                df_valid['magpsf'].astype(float).values,
                df_valid['sigmapsf'].astype(float).values,
                df_valid['magnr'].astype(float).values,
                df_valid['sigmagnr'].astype(float).values,
                df_valid['isdiffpos'].values
            )
        ]
)

df_valid['dc_flux'] = dc_flux
df_valid['dc_sigflux'] = dc_sigflux


for filt in np.unique(df['fid']):
    mask = df_valid['fid'] == filt
    sub = df_valid[mask]
    plt.errorbar(
        sub['jd'].apply(lambda x: x - 2400000.5),
        sub['dc_flux']*1e3,#*1e7/3, 
        sub['dc_sigflux']*1e3,#*1e7/3, 
        ls='', 
        marker='o',
        label=filt
    )
plt.legend()

plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Apparent DC flux (millijanksy)');


# Data by days 

In [ ]:
df['mjd'] = df['jd'].apply(lambda x: x - 2400000.5)

In [ ]:
# Convert 'mjd' to integer to remove fractional part
df['mjd'] = df['mjd'].astype(int)
df2 = df.groupby('mjd')

In [ ]:
df2.size()

In [ ]:
counts_per_day = df2.count()
"""#sum_per_day = df2.sum()

median_per_day = df2.median()
min_per_day = df2.min()
max_per_day = df2.max()
std_per_day = df2.std()
variance_per_day = df2.var()
first_per_day = df2.first()
last_per_day = df2.last()"""
counts_per_day

In [ ]:
df2

In [ ]:
# Assuming df2 contains the DataFrame grouped by 'mjd'
# Calculate the average of 'magpsf' for each day
df_mod = df2['magpsf'].mean().reset_index()

# Resetting index will convert the 'mjd' index to a column and reset the row index


In [ ]:
df_mod